In [1]:
import numpy as np
import pandas as pd

##### (Skeleton using chunks)

### Load the data

In [2]:
DATA_PATH = './data/'
FILE2016 = DATA_PATH + 'quotes-2016.json.bz2'
df_base = pd.read_json(FILE2016, lines=True, compression='bz2', nrows=10000)

### Create a copy to work with

In [3]:
df = df_base.copy()
df

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-09-06-000292,... I feel like I was champion long before I l...,None,[],2016-09-06 20:54:45,2,"[[None, 0.6877], [John Waters, 0.3123]]",[http://onlineathens.com/breaking-news/2016-09...,E
3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
...,...,...,...,...,...,...,...,...,...
9995,2016-08-06-031503,It's a great way to take students out into the...,Greg Schneider,[Q5606223],2016-08-06 11:00:00,1,"[[Greg Schneider, 0.7712], [None, 0.2288]]",[http://greensburgdailynews.com/news/local_new...,E
9996,2016-09-01-065324,"It's a happy accident, to me, that nothing has...",Adam Brody,[Q294372],2016-09-01 15:12:37,1,"[[Adam Brody, 0.4867], [None, 0.438], [Leighto...",[http://www.huffingtonpost.com/2016/09/01/adam...,E
9997,2016-07-27-070749,It's a joyous occasion and I'd like to think a...,Bill Miller,"[Q14951130, Q16187209, Q16732604, Q19325928, Q...",2016-07-27 02:43:37,1,"[[Bill Miller, 0.784], [None, 0.216]]",[http://wabi.tv/2016/07/26/22nd-annual-old-fas...,E
9998,2016-08-15-052447,It's a little bit better than yesterday in som...,None,[],2016-08-15 11:00:00,1,"[[None, 0.6434], [Larry Richard, 0.3566]]",[http://iberianet.com/news/still-rescues-in-th...,E


### Check uniqueness of the keys

In [4]:
print(df.quoteID.is_unique)
df = df.drop_duplicates(subset=['quoteID'], keep='first')

True


In [5]:
def process_chunk_uniqueid(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    print(chunk.quoteID.is_unique, '\n')
    if not(chunk.quoteID.is_unique):
        chunk = chunk.drop_duplicates(subset=['quoteID'], keep='first')

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        process_chunk_uniqueid(chunk)

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 100

KeyboardInterrupt: 

In [6]:
process_chunk_uniqueid(df)

Processing chunk with 10000 rows
True 



### Remove `Nan` quotes 

In [ ]:
df[df['quotation']=='None'].index

### Remove `None` speakers

#### Automated using chunks

In [7]:
def process_chunk_removeNone(chunk):
    tot_length = 0
    print(f'Processing chunk with {len(chunk)} rows')
    chunk = chunk.drop(chunk[chunk['speaker']=='None'].index)
    chunk = chunk.reset_index(drop=True)
    tot_length = tot_length + len(chunk)
    print('Length of the chunk is now', len(chunk), 'rows\n')
    return tot_length
    

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        length_df = process_chunk_removeNone(chunk)

Processing chunk with 10000 rows
Length of the chunk is now 6530 rows

Processing chunk with 10000 rows
Length of the chunk is now 6523 rows

Processing chunk with 10000 rows
Length of the chunk is now 6609 rows



KeyboardInterrupt: 

#### Test over the downloaded dataframe

In [8]:
l = process_chunk_removeNone(df)
df
#don't understand why the output is not of 6530 rows :(

Processing chunk with 10000 rows
Length of the chunk is now 6530 rows



,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-09-06-000292,... I feel like I was champion long before I l...,None,[],2016-09-06 20:54:45,2,"[[None, 0.6877], [John Waters, 0.3123]]",[http://onlineathens.com/breaking-news/2016-09...,E
3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
...,...,...,...,...,...,...,...,...,...
9995,2016-08-06-031503,It's a great way to take students out into the...,Greg Schneider,[Q5606223],2016-08-06 11:00:00,1,"[[Greg Schneider, 0.7712], [None, 0.2288]]",[http://greensburgdailynews.com/news/local_new...,E
9996,2016-09-01-065324,"It's a happy accident, to me, that nothing has...",Adam Brody,[Q294372],2016-09-01 15:12:37,1,"[[Adam Brody, 0.4867], [None, 0.438], [Leighto...",[http://www.huffingtonpost.com/2016/09/01/adam...,E
9997,2016-07-27-070749,It's a joyous occasion and I'd like to think a...,Bill Miller,"[Q14951130, Q16187209, Q16732604, Q19325928, Q...",2016-07-27 02:43:37,1,"[[Bill Miller, 0.784], [None, 0.216]]",[http://wabi.tv/2016/07/26/22nd-annual-old-fas...,E
9998,2016-08-15-052447,It's a little bit better than yesterday in som...,None,[],2016-08-15 11:00:00,1,"[[None, 0.6434], [Larry Richard, 0.3566]]",[http://iberianet.com/news/still-rescues-in-th...,E


#### Applying to a portion of the dataframe (to compare with the cell above)

In [9]:
df_speaker = df.drop(df[df['speaker']=='None'].index)
df_speaker = df_speaker.reset_index(drop=True)
df_speaker

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
3,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
4,2016-02-12-000118,[ It ] would give us more flexibility in terms...,Bob McDonald,"[Q41948618, Q4933333]",2016-02-12 18:30:52,1,"[[Bob McDonald, 0.9298], [None, 0.0702]]",[http://federalnewsradio.com/ses/2016/02/ses-m...,E
...,...,...,...,...,...,...,...,...,...
6525,2016-08-22-053477,It's a great location; you're 15 minutes outsi...,Jose Cruz,"[Q5939096, Q6291967]",2016-08-22 15:09:19,1,"[[Jose Cruz, 0.7555], [None, 0.2445]]",[http://northjersey.com/news/east-rutherford-a...,E
6526,2016-08-06-031503,It's a great way to take students out into the...,Greg Schneider,[Q5606223],2016-08-06 11:00:00,1,"[[Greg Schneider, 0.7712], [None, 0.2288]]",[http://greensburgdailynews.com/news/local_new...,E
6527,2016-09-01-065324,"It's a happy accident, to me, that nothing has...",Adam Brody,[Q294372],2016-09-01 15:12:37,1,"[[Adam Brody, 0.4867], [None, 0.438], [Leighto...",[http://www.huffingtonpost.com/2016/09/01/adam...,E
6528,2016-07-27-070749,It's a joyous occasion and I'd like to think a...,Bill Miller,"[Q14951130, Q16187209, Q16732604, Q19325928, Q...",2016-07-27 02:43:37,1,"[[Bill Miller, 0.784], [None, 0.216]]",[http://wabi.tv/2016/07/26/22nd-annual-old-fas...,E


In [10]:
print('In total,', len(df)-len(df_speaker), 'rows had no speaker and were thus removed')

In total, 3470 rows had no speaker and were thus removed


### Remove speakers for which the probability is smaller than a threshold

#### Automated fashion

In [11]:
def process_chunk_probas(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    #Define threshold (50%)
    threshold_probas = 0.5
    
    #Gather the first probability for each row
    string_probas, nber_probas = [], []
    string_probas = [chunk['probas'][i][0][1] for i in chunk.index]
    nber_probas = list(map(float, string_probas))
    series_probas = pd.Series(nber_probas, dtype='float64', index=chunk.index)
    
    #Check if the probability is larger than the threshold, if not remove corresponding index
    chunk = chunk.drop(series_probas[series_probas < threshold_probas].index)
    chunk = chunk.reset_index(drop=True)
    
    print('Length of the chunk is now', len(chunk), 'rows\n')

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        process_chunk_probas(chunk)

Processing chunk with 10000 rows
Length of the chunk is now 9484 rows

Processing chunk with 10000 rows
Length of the chunk is now 9455 rows

Processing chunk with 10000 rows
Length of the chunk is now 9456 rows

Processing chunk with 10000 rows


KeyboardInterrupt: 

##### Test over the downloaded datafame

In [12]:
process_chunk_probas(df)
df
#once again, seems not to work...

Processing chunk with 10000 rows
Length of the chunk is now 9484 rows



,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-09-06-000292,... I feel like I was champion long before I l...,None,[],2016-09-06 20:54:45,2,"[[None, 0.6877], [John Waters, 0.3123]]",[http://onlineathens.com/breaking-news/2016-09...,E
3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
...,...,...,...,...,...,...,...,...,...
9995,2016-08-06-031503,It's a great way to take students out into the...,Greg Schneider,[Q5606223],2016-08-06 11:00:00,1,"[[Greg Schneider, 0.7712], [None, 0.2288]]",[http://greensburgdailynews.com/news/local_new...,E
9996,2016-09-01-065324,"It's a happy accident, to me, that nothing has...",Adam Brody,[Q294372],2016-09-01 15:12:37,1,"[[Adam Brody, 0.4867], [None, 0.438], [Leighto...",[http://www.huffingtonpost.com/2016/09/01/adam...,E
9997,2016-07-27-070749,It's a joyous occasion and I'd like to think a...,Bill Miller,"[Q14951130, Q16187209, Q16732604, Q19325928, Q...",2016-07-27 02:43:37,1,"[[Bill Miller, 0.784], [None, 0.216]]",[http://wabi.tv/2016/07/26/22nd-annual-old-fas...,E
9998,2016-08-15-052447,It's a little bit better than yesterday in som...,None,[],2016-08-15 11:00:00,1,"[[None, 0.6434], [Larry Richard, 0.3566]]",[http://iberianet.com/news/still-rescues-in-th...,E


#### Applying to a portion of the dataframe

In [13]:
#Define threshold (50%)
threshold_probas = 0.5

#Gather the first probability for each row
string_probas = [df['probas'][i][0][1] for i in range(len(df))]
nber_probas = list(map(float, string_probas))
series_probas = pd.Series(nber_probas)

#Check if the probability is larger than the threshold, if not remove the row
df_probas = df.drop(series_probas[series_probas < threshold_probas].index)
df_probas = df_probas.reset_index(drop=True)
df_probas

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-09-06-000292,... I feel like I was champion long before I l...,None,[],2016-09-06 20:54:45,2,"[[None, 0.6877], [John Waters, 0.3123]]",[http://onlineathens.com/breaking-news/2016-09...,E
3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
...,...,...,...,...,...,...,...,...,...
9479,2016-08-22-053477,It's a great location; you're 15 minutes outsi...,Jose Cruz,"[Q5939096, Q6291967]",2016-08-22 15:09:19,1,"[[Jose Cruz, 0.7555], [None, 0.2445]]",[http://northjersey.com/news/east-rutherford-a...,E
9480,2016-08-06-031503,It's a great way to take students out into the...,Greg Schneider,[Q5606223],2016-08-06 11:00:00,1,"[[Greg Schneider, 0.7712], [None, 0.2288]]",[http://greensburgdailynews.com/news/local_new...,E
9481,2016-07-27-070749,It's a joyous occasion and I'd like to think a...,Bill Miller,"[Q14951130, Q16187209, Q16732604, Q19325928, Q...",2016-07-27 02:43:37,1,"[[Bill Miller, 0.784], [None, 0.216]]",[http://wabi.tv/2016/07/26/22nd-annual-old-fas...,E
9482,2016-08-15-052447,It's a little bit better than yesterday in som...,None,[],2016-08-15 11:00:00,1,"[[None, 0.6434], [Larry Richard, 0.3566]]",[http://iberianet.com/news/still-rescues-in-th...,E


In [14]:
print('In total,', len(df)-len(df_probas), 'speakers had a probality smaller than', threshold_probas, 'and were thus removed')

In total, 516 speakers had a probality smaller than 0.5 and were thus removed


#### Think to other scenarios

In [ ]:
#If all speakers are women but with proba < threshold, keep the row ?
#might cause some problem for interpretation (country of origin of the speaker)


#### Additional notes Lavi 1.11.

- I don't think we should keep the speakers if proba < threshold, because like you state above, this will create additional problems

Possible additional cleaning may include:
- checking nan values 
- verify absurd QuoteID or QID 
- make sure quotes are non empty

